In [5]:
# import the required libraries
import docx2txt
import os
import re
import csv
import pandas as pd
import numpy as np
import act1

In [6]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_file = os.path.join(folderlink+input_folder, 'Repertoriummetoudepersoonsnummers1.docx')
folder_output = 'output//'
output_txt = folderlink+folder_output+'output.txt'
output_csv = folderlink+folder_output+'output_file.csv'

In [7]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [8]:
# Converting the docx to a text file and remove all unecessary rows.

# Use docx2txt library to extract text from .docx file
text = docx2txt.process(input_file)

# Remove excessive whitespaces
data = ' '.join(text.split())

Manual updates in the dataset:

1.  FROM: "N.N. "de oude vicarius">pred. Lichtenvoorde 1602 tot 1615.<20871>"
    TO: "N.N. "de oude vicarius" pred. Lichtenvoorde 1602 tot 1615.<20871>" 
    REASON: since it contained the > character which is used to define ID fields. 
 
2.  FROM: "Bosch; Cornelius Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713,;overl. 28 maart 1715.<1185>"
    TO:Bosch; Cornelius
Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713, overl. 28 maart 1715.<1185>
    REASON: Since it splitted the string based on ",;overl"

3.  FROM: Leeuwen, van, Cornelis [z.v. Cornelis]
Geb. Hazerswoude 1611; pred. Boskoop en Middelburg 1637, overl. 1681.<5778>
    TO: Leeuwen, van; Cornelis [z.v. Cornelis]
Geb. Hazerswoude 1611; pred. Boskoop en Middelburg 1637, overl. 1681.<5778>
    REASON: there was no ; between the name and surname (including infix)

4.  FROM: Peenen, van, Marcus
Gedoopt Leiden 31 aug. 1642; pred. Koudekerk aan den Rijn 2 sept. 1668, Leiden 1680, begraven 1 febr. 1696.<7388>
    TO: Peenen, van; Marcus
Gedoopt Leiden 31 aug. 1642; pred. Koudekerk aan den Rijn 2 sept. 1668, Leiden 1680, begraven 1 febr. 1696.<7388>
    REASON: there was no ; between the name and surname (including infix)

5.  FROM: Knuyt, de (Kuntius), Elias
Geb. Middelburg yyy; pred. Oude Niedorp en Veenhuizen (NH) mei 1628, Sint Annaland 1630, Westkapelle 7 maart 1641, overl. --of vertrokken?-- 1642.<5381>
    TO: Knuyt, de (Kuntius); Elias
Geb. Middelburg yyy; pred. Oude Niedorp en Veenhuizen (NH) mei 1628, Sint Annaland 1630, Westkapelle 7 maart 1641, overl. --of vertrokken?-- 1642.<5381>
    REASON: there was no ; between the name and surname (including infix)

6.  FROM: Leonardis, de, Paulus
Geb. Keulen yyy; pred. Bacharach (Pfalz) 16.., Kampen 1620, overl. 1649.<5836>
    TO: Leonardis, de; Paulus
Geb. Keulen yyy; pred. Bacharach (Pfalz) 16.., Kampen 1620, overl. 1649.<5836>
    REASON: there was no ; between the name and surname (including infix)

7.  FROM: Tamerus; Henricus
Geb. xxx ca. 1540; voorheen Lutheraan;--onwettig pred. Heteren en Randwijk ca. 1600-1602; pred. Eethen en Meeuwen 1606, Doeveren, Gansoyen en Genderen 1610, afgezet als remonstrant 1619; overl. Heusden.<21206>
    TO: Tamerus; Henricus
Geb. xxx ca. 1540; voorheen Lutheraan; onwettig pred. Heteren en Randwijk ca. 1600-1602; pred. Eethen en Meeuwen 1606, Doeveren, Gansoyen en Genderen 1610, afgezet als remonstrant 1619; overl. Heusden.<21206>
    REASON: -- conflicted created a line break. 
    
8. FROM Aitton; Rijk Otto [z.v. Hendrik Arnold]
Geb. Zwolle 27 maart 1790; pred. Zuilichem + Nieuwaal 3 maart 1811, --legerpred. 1815, garnizoenspred. Oostende (Vlaanderen)-- Aalten 11 mei 1817, Hooge Zwaluwe 9 april 1826, Monster 4 mei 1828, Zevenbergen 5 april 1840, emer. 1855; overl. 4 aug. 1863.<130>
    TO: Aitton; Rijk Otto [z.v. Hendrik Arnold]
Geb. Zwolle 27 maart 1790; pred. Zuilichem + Nieuwaal 3 maart 1811, legerpred. 1815, garnizoenspred. Oostende (Vlaanderen), pred. Aalten 11 mei 1817, Hooge Zwaluwe 9 april 1826, Monster 4 mei 1828, Zevenbergen 5 april 1840, emer. 1855; overl. 4 aug. 1863.<130>
    REASON: second pred. was missing.




In [9]:
data = data.replace('(Vlaanderen)§§', '(Vlaanderen)§§, pred.').replace('overl. 28 maart 1715.<1185>', ' overl. 28 maart 1715.<1185>').replace('>pred.', ' pred.').replace('Leeuwen, van, Cornelis [z.v. Cornelis]','Leeuwen, van; Cornelis [z.v. Cornelis]',).replace('Peenen, van, Marcus','Peenen, van; Marcus').replace('Knuyt, de (Kuntius), Elias','Knuyt, de (Kuntius); Elias').replace('Leonardis, de, Paulus','Leonardis, de; Paulus').replace('Lutheraan;§§onwettig','Lutheraan; onwettig')
data = data.replace('§', '-')

In [10]:
# Replace semicolons with newlines and add semicolons around < and > since these identify the IDs
data = data.replace(';', ';\n').replace(';\n ', '; ').replace('>', '>;\n ').replace('<', ';<')
lines = data.split('\n')

lines = [line for line in lines if not line.startswith('; ;') and not line.startswith('; ')]
data = '\n'.join(lines)
lines = data.strip().split('\n')
data = '\n'.join([line.lstrip() for line in lines])


In [ ]:
def replace_pred_count(string):
    count = 0
    result = ""
    position = string.find(" pred.")

    while position != -1:
        result += string[:position] + "zx"+ str(count) + " pred."
        string = string[position + len(" pred."):]
        count += 1
        position = string.find(" pred.")

    result += string

    return result

In [11]:
lines = data.split('\n')

with open(output_txt, "w", encoding='utf-8') as file:
    for line in lines:
        result_string = replace_pred_count(line)
        file.write(result_string + "\n")

In [12]:
columns = ("Geb.", 
 "zx0 pred.",
 "zx1 pred.",
 "zx2 pred.",
 "zx3 pred.",
 "zx4 pred.",
 "overl.", 
 "Gedoopt", 
 "legerpred.",
 "pastoor",
 "garnizoenspred.",
 "emer.",
 "begraven",
 "conrector",
 " rector",
 "monnik",           
 "schoolmeester",
 "hoogleraar",
 "chirurgijn",
 "praeceptor",
 "ziekentrooster",
 "vlootpred.",
 "legerpred.",
 "ambassadepred."           
)          
    

In [13]:
for column in columns:
    with open(output_txt, 'r', encoding='utf-8') as f:
        lines = f.readlines()        
    with open(output_txt,'w', encoding='utf-8') as f:
        for line in lines:
            if "; "+column in line:
                f.write(line)
            elif column in line:
                line = line.replace(column, ";"+column)
                f.write(line)
            else:
                f.write(line)

In [14]:
# Define the headers for the output file
headers = ['surname_temp', 'first_name', 'Field1', 'Field2', 'Field3', 'Field4', 'Field5', 'Field6', 'Field7', 'Field8', 'Field9', 'Field10', 'Field11','Field12','Field13','Field14','Field15','Field16','Field17','Field18']

with open(output_txt, 'r', encoding='utf-8') as infile, open(output_csv, 'w', newline='', encoding='utf-8' ) as outfile:
    reader = csv.reader(infile, delimiter=';')
    writer = csv.writer(outfile, delimiter=';')
    
    # Write the headers to the output file
    writer.writerow(headers)

    # Loop through each row in the input file and write it to the output file with 12 fields
    for row in reader:
        # Create a new row with 12 fields by extending the current row with empty values
        new_row = row + [''] * (12 - len(row))
        writer.writerow(new_row)

In [15]:
with open(output_csv, 'r') as file:
    content = file.read()

modified_content = content.replace("§ ", "-")

with open(output_csv, 'w') as file:
    file.write(modified_content)

In [16]:
df = pd.read_csv(output_csv, sep=';', encoding='utf-8')

In [17]:
#In the file all IDs are stored between < and > therefore 
for column in df.columns:
    # Check if any value in the column contains '<'
    if df[column].astype(str).str.contains('<').any():
        # Copy the values containing '<' to the 'test' column
        df.loc[df[column].astype(str).str.contains('<'), 'drc_id'] = df[column]


In [18]:
df['drc_id'] = df['drc_id'].str.replace('>', '').str.replace('<','')

In [19]:
for column in columns:
    df[column] = df.apply(lambda row: row[row.astype(str).str.contains(column)].iloc[0] if any(row.astype(str).str.contains(column)) else '', axis=1)

In [20]:
df.head()

,surname_temp,first_name,Field1,Field2,Field3,Field4,Field5,Field6,Field7,Field8,Field9,Field10,Field11,Field12,Field13,Field14,Field15,Field16,Field17,Field18,drc_id,Geb.,zx0 pred.,zx1 pred.,zx2 pred.,zx3 pred.,zx4 pred.,overl.,Gedoopt,legerpred.,pastoor,garnizoenspred.,emer.,begraven,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpred.,ambassadepred.
0,"Aalburg, van",Johannes,Geb. Zierikzee ca. 1717,NaN,"zx0 pred. Oudkarspel 30 juni 1743,",overl. 14 maart 1777.,<1>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Geb. Zierikzee ca. 1717,"zx0 pred. Oudkarspel 30 juni 1743,",,,,,overl. 14 maart 1777.,,,,,,,,,,,,,,,,
1,Aalst,Wilhelmus,Gedoopt Biggekerke 5 jan. 1664,NaN,"zx0 pred. Aardenburg 22 mei 1695,",overl. 19 dec. 1700.,<4>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,,"zx0 pred. Aardenburg 22 mei 1695,",,,,,overl. 19 dec. 1700.,Gedoopt Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,
2,"Aalst, van",Cornelius,Geb. Castricum ca. 1686,ambassadepred. in Parijs maart tot dec. 1715,NaN,"zx0 pred. Kalslagen ber. 21 febr. 1717,",emer. 1751,overl. Amsterdam 27 aug. 1756.,<2>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Geb. Castricum ca. 1686,"zx0 pred. Kalslagen ber. 21 febr. 1717,",,,,,overl. Amsterdam 27 aug. 1756.,,,,,emer. 1751,,,,,,,,,,,ambassadepred. in Parijs maart tot dec. 1715
3,"Aalst, van",Gerardus,Geb. xxx sept. 1678,NaN,"zx0 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715,",emer. 1755,overl. 29 juni 1759.,<3>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Geb. xxx sept. 1678,"zx0 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715,",,,,,overl. 29 juni 1759.,,,,,emer. 1755,,,,,,,,,,,
4,Aalstius,Henricus,Geb. 's-Hertogenbosch (?) yyy,NaN,"zx0 pred. Castricum en Heemskerk nov. 1700,",emer. sept. 1733,overl. Limmen 15 maart 1736.,<5>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Geb. 's-Hertogenbosch (?) yyy,"zx0 pred. Castricum en Heemskerk nov. 1700,",,,,,overl. Limmen 15 maart 1736.,,,,,emer. sept. 1733,,,,,,,,,,,


In [21]:
dft = df

In [22]:
df['original_input'] = df['surname_temp'].fillna('') + df['first_name'].fillna('') + df['Field1'].fillna('') + df['Field2'].fillna('')+ df['Field3'].fillna('')+ df['Field4'].fillna('')+ df['Field5'].fillna('')+ df['Field6'].fillna('')+ df['Field7'].fillna('')+ df['Field8'].fillna('')+ df['Field9'].fillna('')+ df['Field10'].fillna('')+ df['Field11'].fillna('')+ df['Field12'].fillna('')+ df['Field13'].fillna('')+ df['Field14'].fillna('')+ df['Field15'].fillna('')+ df['Field16'].fillna('')+ df['Field17'].fillna('')+ df['Field18'].fillna('')


In [23]:
df['original_input'] = df['original_input'].str.replace('zx0 pred. ', ' pred. ').str.replace('zx1 pred.',' pred. ').str.replace('zx2 pred.',' pred. ').str.replace('zx3 pred.',' pred. ').str.replace('zx4 pred.',' pred. ')

In [24]:
df.head()

,surname_temp,first_name,Field1,Field2,Field3,Field4,Field5,Field6,Field7,Field8,Field9,Field10,Field11,Field12,Field13,Field14,Field15,Field16,Field17,Field18,drc_id,Geb.,zx0 pred.,zx1 pred.,zx2 pred.,zx3 pred.,zx4 pred.,overl.,Gedoopt,legerpred.,pastoor,garnizoenspred.,emer.,begraven,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpred.,ambassadepred.,original_input
0,"Aalburg, van",Johannes,Geb. Zierikzee ca. 1717,NaN,"zx0 pred. Oudkarspel 30 juni 1743,",overl. 14 maart 1777.,<1>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Geb. Zierikzee ca. 1717,"zx0 pred. Oudkarspel 30 juni 1743,",,,,,overl. 14 maart 1777.,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>"
1,Aalst,Wilhelmus,Gedoopt Biggekerke 5 jan. 1664,NaN,"zx0 pred. Aardenburg 22 mei 1695,",overl. 19 dec. 1700.,<4>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,,"zx0 pred. Aardenburg 22 mei 1695,",,,,,overl. 19 dec. 1700.,Gedoopt Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>"
2,"Aalst, van",Cornelius,Geb. Castricum ca. 1686,ambassadepred. in Parijs maart tot dec. 1715,NaN,"zx0 pred. Kalslagen ber. 21 febr. 1717,",emer. 1751,overl. Amsterdam 27 aug. 1756.,<2>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Geb. Castricum ca. 1686,"zx0 pred. Kalslagen ber. 21 febr. 1717,",,,,,overl. Amsterdam 27 aug. 1756.,,,,,emer. 1751,,,,,,,,,,,ambassadepred. in Parijs maart tot dec. 1715,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>"
3,"Aalst, van",Gerardus,Geb. xxx sept. 1678,NaN,"zx0 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715,",emer. 1755,overl. 29 juni 1759.,<3>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,Geb. xxx sept. 1678,"zx0 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715,",,,,,overl. 29 juni 1759.,,,,,emer. 1755,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>"
4,Aalstius,Henricus,Geb. 's-Hertogenbosch (?) yyy,NaN,"zx0 pred. Castricum en Heemskerk nov. 1700,",emer. sept. 1733,overl. Limmen 15 maart 1736.,<5>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,Geb. 's-Hertogenbosch (?) yyy,"zx0 pred. Castricum en Heemskerk nov. 1700,",,,,,overl. Limmen 15 maart 1736.,,,,,emer. sept. 1733,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>"


In [25]:
for column in columns:
    df[column] = df[column].str.replace(column, '')

In [26]:
df['name_info_family'] =df['first_name'] .str.extract(r'\[(.*?)\]')


In [27]:
df['alternative_name'] =df['surname_temp'] .str.extract(r'\((.*?)\)')


In [28]:
columns_rename = {
    'Geb.': 'birth', 
    'overl.': 'death', 
    'Gedoopt':'baptized', 
    'legerpred.':'legerpredikant',
    'pastoor':'pastoor',
    'garnizoenspred.':'garnizoenspredikant',
    "emer.":'emeritus_status',
    "begraven":'burried',
    "conrector":'conrector',
    " rector":'rector',
    "monnik":'monnik',           
    "schoolmeester":'schoolmeester',
    "hoogleraar":'hoogleraar',
    "chirurgijn":'chirurgijn',
    "praeceptor":'praeceptor',
    "ziekentrooster":'ziekentrooster',
    "vlootpred.":'vlootpredikant',
    "legerpred.":'legerpredikant',
    "ambassadepred.":'ambassadepredikant'}

In [29]:
# Rename the columns
df = df.rename(columns=columns_rename)
new_columns = list(columns_rename.values())


In [30]:
array_drop = [i for i in range(1, 19)]
for dropid in array_drop:
    column_dropid = 'Field'+str(dropid)
    df = df.drop(column_dropid, axis=1)

In [31]:
def extract_year(text):
    match = re.search(r'\d{4}', text)
    if match:
        return match.group(0)
    else:
        return None

In [32]:
df['minister'] = df['zx0 pred.']+ ','+df['zx1 pred.']+ ','+df['zx2 pred.']+ ','+df['zx3 pred.']+ ','+df['zx4 pred.']

In [33]:
df = df.drop(['zx0 pred.','zx1 pred.','zx2 pred.','zx3 pred.','zx4 pred.'], axis=1)

In [34]:
function_year = [word for word in new_columns if word != 'minister']

print(function_year)

['birth', 'death', 'baptized', 'legerpredikant', 'pastoor', 'garnizoenspredikant', 'emeritus_status', 'burried', 'conrector', 'rector', 'monnik', 'schoolmeester', 'hoogleraar', 'chirurgijn', 'praeceptor', 'ziekentrooster', 'vlootpredikant', 'ambassadepredikant']


In [35]:
df.head()

,surname_temp,first_name,drc_id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,alternative_name,minister
0,"Aalburg, van",Johannes,1,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,"
1,Aalst,Wilhelmus,4,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,"
2,"Aalst, van",Cornelius,2,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,"
3,"Aalst, van",Gerardus,3,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, ,,,,"
4,Aalstius,Henricus,5,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,"


In [36]:
for year in function_year:
    fld_year = year +'_year'
    df[fld_year] = df[year].apply(lambda x: extract_year(x))    

In [37]:
for year_accu in function_year:
    accu_fld_year = year_accu +'_year_accuracy'
    df[accu_fld_year] = ''
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Check if the string contains "ca." (case-insensitive)
        if 'ca.' in row[year_accu].lower():
            # If found, set the value of the "accuracy" column to "circa"
            df.at[index, accu_fld_year] = 'circa'
            
    

In [38]:
for remark in function_year:
    remarks_field = 'remarks_'+remark
    df[remarks_field] = df[remark]

In [39]:
df.head()

,surname_temp,first_name,drc_id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,alternative_name,minister,birth_year,death_year,baptized_year,legerpredikant_year,pastoor_year,garnizoenspredikant_year,emeritus_status_year,burried_year,conrector_year,rector_year,monnik_year,schoolmeester_year,hoogleraar_year,chirurgijn_year,praeceptor_year,ziekentrooster_year,vlootpredikant_year,ambassadepredikant_year,birth_year_accuracy,death_year_accuracy,baptized_year_accuracy,legerpredikant_year_accuracy,pastoor_year_accuracy,garnizoenspredikant_year_accuracy,emeritus_status_year_accuracy,burried_year_accuracy,conrector_year_accuracy,rector_year_accuracy,monnik_year_accuracy,schoolmeester_year_accuracy,hoogleraar_year_accuracy,chirurgijn_year_accuracy,praeceptor_year_accuracy,ziekentrooster_year_accuracy,vlootpredikant_year_accuracy,ambassadepredikant_year_accuracy,remarks_birth,remarks_death,remarks_baptized,remarks_legerpredikant,remarks_pastoor,remarks_garnizoenspredikant,remarks_emeritus_status,remarks_burried,remarks_conrector,remarks_rector,remarks_monnik,remarks_schoolmeester,remarks_hoogleraar,remarks_chirurgijn,remarks_praeceptor,remarks_ziekentrooster,remarks_vlootpredikant,remarks_ambassadepredikant
0,"Aalburg, van",Johannes,1,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,
1,Aalst,Wilhelmus,4,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,
2,"Aalst, van",Cornelius,2,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715
3,"Aalst, van",Gerardus,3,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,
4,Aalstius,Henricus,5,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,


In [40]:
months =(" januari ", 
 " februari ",
 " maart ",
 " april ",
 " mei ",
 " juni ",
 " juli ",
 " augustus ",
 " september ",
 " oktober ",
 " november ",           
 " december ",
 "jan. ",
 "feb. ",
 "mrt. ",
 "apr. ",
 "jun. ",
 "jul. ",
 "aug. ",
 "sept. ",
 "sep. ",
 "okt. ",
 "nov. ",
 "dec. ",
 "yyy",
 "xxx",
 "ca.",
 "febr."        
) 


In [41]:
for column_strip in function_year:   
    for month in months:
        df[column_strip] = df[column_strip].str.replace(month, '')


In [42]:
for column_strip in function_year:
    df[column_strip] = df[column_strip].apply(lambda x: re.sub(r'[\d\.]', '', x))

In [43]:
df.head()

,surname_temp,first_name,drc_id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,alternative_name,minister,birth_year,death_year,baptized_year,legerpredikant_year,pastoor_year,garnizoenspredikant_year,emeritus_status_year,burried_year,conrector_year,rector_year,monnik_year,schoolmeester_year,hoogleraar_year,chirurgijn_year,praeceptor_year,ziekentrooster_year,vlootpredikant_year,ambassadepredikant_year,birth_year_accuracy,death_year_accuracy,baptized_year_accuracy,legerpredikant_year_accuracy,pastoor_year_accuracy,garnizoenspredikant_year_accuracy,emeritus_status_year_accuracy,burried_year_accuracy,conrector_year_accuracy,rector_year_accuracy,monnik_year_accuracy,schoolmeester_year_accuracy,hoogleraar_year_accuracy,chirurgijn_year_accuracy,praeceptor_year_accuracy,ziekentrooster_year_accuracy,vlootpredikant_year_accuracy,ambassadepredikant_year_accuracy,remarks_birth,remarks_death,remarks_baptized,remarks_legerpredikant,remarks_pastoor,remarks_garnizoenspredikant,remarks_emeritus_status,remarks_burried,remarks_conrector,remarks_rector,remarks_monnik,remarks_schoolmeester,remarks_hoogleraar,remarks_chirurgijn,remarks_praeceptor,remarks_ziekentrooster,remarks_vlootpredikant,remarks_ambassadepredikant
0,"Aalburg, van",Johannes,1,Zierikzee,,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,
1,Aalst,Wilhelmus,4,,,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,
2,"Aalst, van",Cornelius,2,Castricum,Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715
3,"Aalst, van",Gerardus,3,,,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,
4,Aalstius,Henricus,5,'s-Hertogenbosch (?),Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,


In [44]:
df['surname_temp']= df['surname_temp'].str.replace(r'\(.*\)', '', regex=True)
df['first_name']= df['first_name'].str.replace(r'\[.*\]', '', regex=True)

In [45]:
df.head()

,surname_temp,first_name,drc_id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,alternative_name,minister,birth_year,death_year,baptized_year,legerpredikant_year,pastoor_year,garnizoenspredikant_year,emeritus_status_year,burried_year,conrector_year,rector_year,monnik_year,schoolmeester_year,hoogleraar_year,chirurgijn_year,praeceptor_year,ziekentrooster_year,vlootpredikant_year,ambassadepredikant_year,birth_year_accuracy,death_year_accuracy,baptized_year_accuracy,legerpredikant_year_accuracy,pastoor_year_accuracy,garnizoenspredikant_year_accuracy,emeritus_status_year_accuracy,burried_year_accuracy,conrector_year_accuracy,rector_year_accuracy,monnik_year_accuracy,schoolmeester_year_accuracy,hoogleraar_year_accuracy,chirurgijn_year_accuracy,praeceptor_year_accuracy,ziekentrooster_year_accuracy,vlootpredikant_year_accuracy,ambassadepredikant_year_accuracy,remarks_birth,remarks_death,remarks_baptized,remarks_legerpredikant,remarks_pastoor,remarks_garnizoenspredikant,remarks_emeritus_status,remarks_burried,remarks_conrector,remarks_rector,remarks_monnik,remarks_schoolmeester,remarks_hoogleraar,remarks_chirurgijn,remarks_praeceptor,remarks_ziekentrooster,remarks_vlootpredikant,remarks_ambassadepredikant
0,"Aalburg, van",Johannes,1,Zierikzee,,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,
1,Aalst,Wilhelmus,4,,,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,
2,"Aalst, van",Cornelius,2,Castricum,Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715
3,"Aalst, van",Gerardus,3,,,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,
4,Aalstius,Henricus,5,'s-Hertogenbosch (?),Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,


In [46]:
df[['surname', 'infix']] = df['surname_temp'].str.split(',', expand=True)

In [47]:
df['first_letter'] = df['first_name'].astype(str).apply(lambda x: x[1])

In [48]:
df.head()

,surname_temp,first_name,drc_id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,alternative_name,minister,birth_year,death_year,baptized_year,legerpredikant_year,pastoor_year,garnizoenspredikant_year,emeritus_status_year,burried_year,conrector_year,rector_year,monnik_year,schoolmeester_year,hoogleraar_year,chirurgijn_year,praeceptor_year,ziekentrooster_year,vlootpredikant_year,ambassadepredikant_year,birth_year_accuracy,death_year_accuracy,baptized_year_accuracy,legerpredikant_year_accuracy,pastoor_year_accuracy,garnizoenspredikant_year_accuracy,emeritus_status_year_accuracy,burried_year_accuracy,conrector_year_accuracy,rector_year_accuracy,monnik_year_accuracy,schoolmeester_year_accuracy,hoogleraar_year_accuracy,chirurgijn_year_accuracy,praeceptor_year_accuracy,ziekentrooster_year_accuracy,vlootpredikant_year_accuracy,ambassadepredikant_year_accuracy,remarks_birth,remarks_death,remarks_baptized,remarks_legerpredikant,remarks_pastoor,remarks_garnizoenspredikant,remarks_emeritus_status,remarks_burried,remarks_conrector,remarks_rector,remarks_monnik,remarks_schoolmeester,remarks_hoogleraar,remarks_chirurgijn,remarks_praeceptor,remarks_ziekentrooster,remarks_vlootpredikant,remarks_ambassadepredikant,surname,infix,first_letter
0,"Aalburg, van",Johannes,1,Zierikzee,,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,,Aalburg,van,J
1,Aalst,Wilhelmus,4,,,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,Aalst,None,W
2,"Aalst, van",Cornelius,2,Castricum,Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,Aalst,van,C
3,"Aalst, van",Gerardus,3,,,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,,Aalst,van,G
4,Aalstius,Henricus,5,'s-Hertogenbosch (?),Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,Aalstius,None,H


In [49]:
df = df.drop(['surname_temp'], axis=1)

In [50]:
# This join field is only created for the linkages with DM should be transfered lateron to there

df['join_name'] = df['surname']+df['first_name']+df['infix'].fillna('')
df['join_name'] = df['join_name'].str.replace("  "," ")

In [51]:
df.head()

,first_name,drc_id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,alternative_name,minister,birth_year,death_year,baptized_year,legerpredikant_year,pastoor_year,garnizoenspredikant_year,emeritus_status_year,burried_year,conrector_year,rector_year,monnik_year,schoolmeester_year,hoogleraar_year,chirurgijn_year,praeceptor_year,ziekentrooster_year,vlootpredikant_year,ambassadepredikant_year,birth_year_accuracy,death_year_accuracy,baptized_year_accuracy,legerpredikant_year_accuracy,pastoor_year_accuracy,garnizoenspredikant_year_accuracy,emeritus_status_year_accuracy,burried_year_accuracy,conrector_year_accuracy,rector_year_accuracy,monnik_year_accuracy,schoolmeester_year_accuracy,hoogleraar_year_accuracy,chirurgijn_year_accuracy,praeceptor_year_accuracy,ziekentrooster_year_accuracy,vlootpredikant_year_accuracy,ambassadepredikant_year_accuracy,remarks_birth,remarks_death,remarks_baptized,remarks_legerpredikant,remarks_pastoor,remarks_garnizoenspredikant,remarks_emeritus_status,remarks_burried,remarks_conrector,remarks_rector,remarks_monnik,remarks_schoolmeester,remarks_hoogleraar,remarks_chirurgijn,remarks_praeceptor,remarks_ziekentrooster,remarks_vlootpredikant,remarks_ambassadepredikant,surname,infix,first_letter,join_name
0,Johannes,1,Zierikzee,,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,,Aalburg,van,J,Aalburg Johannes van
1,Wilhelmus,4,,,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,Aalst,None,W,Aalst Wilhelmus
2,Cornelius,2,Castricum,Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,Aalst,van,C,Aalst Cornelius van
3,Gerardus,3,,,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,,Aalst,van,G,Aalst Gerardus van
4,Henricus,5,'s-Hertogenbosch (?),Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,Aalstius,None,H,Aalstius Henricus


In [52]:
exclude_elements = ['birth', 'death', 'baptized', 'burried']

roles = [item for item in function_year if item not in exclude_elements]

In [53]:
child_role_dfs = []

for role in roles:
    accu_year = role +'_year_accuracy'
    year = role + '_year'
    role_remarks = 'remarks_'+role
    role_df = role
    role_df = df[['drc_id',role,year,accu_year,role_remarks]]
    columns_to_check = [role, year, accu_year, role_remarks]    
    role_df = role_df[role_df[columns_to_check].notna().all(axis=1)]
    new_column_names = {role:'role_place', accu_year : 'role_start_year_accuracy', year : 'role_start_year', role_remarks : 'role_remarks'}
    role_df.rename(columns=new_column_names, inplace=True)
    role_df['role_type'] = role
    child_role_dfs.append(role_df)

child_role = pd.concat(child_role_dfs, ignore_index=True)



In [55]:
column_list = df.columns.tolist()


In [56]:
df['place_birth'] = df['birth']
df['place_death'] = df['death']
df['place_baptized'] = df['baptized']
df['place_burried'] = df['burried']

In [58]:
drc_parent = df[['drc_id','first_name', 'infix', 'surname', 'first_letter', 'place_birth', 'place_death', 'place_baptized', 'place_burried', 'original_input', 'name_info_family',  'birth_year', 'death_year', 'baptized_year', 'burried_year', 'birth_year_accuracy', 'death_year_accuracy', 'baptized_year_accuracy', 'burried_year_accuracy', 'join_name']]
drc_parent.to_csv(folderlink+folder_output+'drc_data.csv', sep=';', encoding='utf-8', index=False)

In [59]:
drc_child_alt_name = df[['drc_id','alternative_name']]
drc_child_alt_name.to_csv(folderlink+folder_output+'drc_child_alt_name.csv', sep=';', encoding='utf-8', index=False)

From here we are going to make the parent child relationship 

Here the minister data is set as a child. 

In [60]:
subset_pred = df[['drc_id', 'minister','first_letter','join_name']] #,'original_input']]

In [61]:
subset_pred.head()

,drc_id,minister,first_letter,join_name
0,1,"Oudkarspel 30 juni 1743, ,,,,",J,Aalburg Johannes van
1,4,"Aardenburg 22 mei 1695, ,,,,",W,Aalst Wilhelmus
2,2,"Kalslagen ber. 21 febr. 1717, ,,,,",C,Aalst Cornelius van
3,3,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, West-Zaandam 4 aug. 1715, ,,,,",G,Aalst Gerardus van
4,5,"Castricum en Heemskerk nov. 1700, ,,,,",H,Aalstius Henricus


In [62]:
df_expanded = subset_pred.assign(minister=subset_pred['minister'].str.split(','))

# Explode the 'pred.' column to create separate rows for each item
df_expanded = df_expanded.explode('minister')

In [63]:
df_expanded.head()

,drc_id,minister,first_letter,join_name
0,1,Oudkarspel 30 juni 1743,J,Aalburg Johannes van
0,1,,J,Aalburg Johannes van
0,1,,J,Aalburg Johannes van
0,1,,J,Aalburg Johannes van
0,1,,J,Aalburg Johannes van


In [64]:
df_filtered = df_expanded[['drc_id','first_letter', 'minister','join_name']]#,'original_input']]

In [65]:
childs = df_filtered[(df_filtered["minister"] !=" ") & (df_filtered["minister"] !="")]

In [66]:
childs.head()

,drc_id,first_letter,minister,join_name
0,1,J,Oudkarspel 30 juni 1743,Aalburg Johannes van
1,4,W,Aardenburg 22 mei 1695,Aalst Wilhelmus
2,2,C,Kalslagen ber. 21 febr. 1717,Aalst Cornelius van
3,3,G,Vuren en Dalem 10 aug. 1704,Aalst Gerardus van
3,3,G,Sommelsdijk 13 juni 1706,Aalst Gerardus van


In [67]:
childs['minister_year'] = childs['minister'].apply(lambda x: extract_year(x))

C:\Windows\Temp\ipykernel_5664\3394215768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['minister_year'] = childs['minister'].apply(lambda x: extract_year(x))


In [68]:
childs['role_start_year_accuracy'] = ''
childs.loc[childs['minister'].str.contains('ca\.', case=False), 'role_start_year_accuracy'] = 'circa'

C:\Windows\Temp\ipykernel_5664\2276370748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['role_start_year_accuracy'] = ''


In [69]:
childs['role_remarks'] = childs['minister']

C:\Windows\Temp\ipykernel_5664\4108002290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['role_remarks'] = childs['minister']


In [70]:
for month in months:
        childs['minister'] = childs['minister'].str.replace(month, '')

childs['minister'] = childs['minister'].apply(lambda x: re.sub(r'[\d\.]', '', x))

C:\Windows\Temp\ipykernel_5664\1973265549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['minister'] = childs['minister'].str.replace(month, '')
C:\Windows\Temp\ipykernel_5664\1973265549.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['minister'] = childs['minister'].apply(lambda x: re.sub(r'[\d\.]', '', x))


In [71]:
childs.head()

,drc_id,first_letter,minister,join_name,minister_year,role_start_year_accuracy,role_remarks
0,1,J,Oudkarspel,Aalburg Johannes van,1743,,Oudkarspel 30 juni 1743
1,4,W,Aardenburg,Aalst Wilhelmus,1695,,Aardenburg 22 mei 1695
2,2,C,Kalslagen ber,Aalst Cornelius van,1717,,Kalslagen ber. 21 febr. 1717
3,3,G,Vuren en Dalem,Aalst Gerardus van,1704,,Vuren en Dalem 10 aug. 1704
3,3,G,Sommelsdijk,Aalst Gerardus van,1706,,Sommelsdijk 13 juni 1706


In [72]:
childs['role_type'] = "predikant"

C:\Windows\Temp\ipykernel_5664\3547419778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs['role_type'] = "predikant"


In [73]:
childs.head()

,drc_id,first_letter,minister,join_name,minister_year,role_start_year_accuracy,role_remarks,role_type
0,1,J,Oudkarspel,Aalburg Johannes van,1743,,Oudkarspel 30 juni 1743,predikant
1,4,W,Aardenburg,Aalst Wilhelmus,1695,,Aardenburg 22 mei 1695,predikant
2,2,C,Kalslagen ber,Aalst Cornelius van,1717,,Kalslagen ber. 21 febr. 1717,predikant
3,3,G,Vuren en Dalem,Aalst Gerardus van,1704,,Vuren en Dalem 10 aug. 1704,predikant
3,3,G,Sommelsdijk,Aalst Gerardus van,1706,,Sommelsdijk 13 juni 1706,predikant


In [74]:
new_column_names_min = {'minister':'role_place', 'minister_year' : 'role_start_year'}
childs.rename(columns=new_column_names_min, inplace=True)

C:\Windows\Temp\ipykernel_5664\4043570210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  childs.rename(columns=new_column_names_min, inplace=True)


In [75]:
childs.head()

,drc_id,first_letter,role_place,join_name,role_start_year,role_start_year_accuracy,role_remarks,role_type
0,1,J,Oudkarspel,Aalburg Johannes van,1743,,Oudkarspel 30 juni 1743,predikant
1,4,W,Aardenburg,Aalst Wilhelmus,1695,,Aardenburg 22 mei 1695,predikant
2,2,C,Kalslagen ber,Aalst Cornelius van,1717,,Kalslagen ber. 21 febr. 1717,predikant
3,3,G,Vuren en Dalem,Aalst Gerardus van,1704,,Vuren en Dalem 10 aug. 1704,predikant
3,3,G,Sommelsdijk,Aalst Gerardus van,1706,,Sommelsdijk 13 juni 1706,predikant


In [76]:
role_minister = childs[['drc_id', 'role_type','role_place','role_start_year','role_start_year_accuracy','role_remarks']]

In [77]:
role_minister.describe()

,drc_id,role_type,role_place,role_start_year,role_start_year_accuracy,role_remarks
count,28871,28871,28871,27212,28871,28871
unique,12497,1,7940,288,2,27141
top,20982,predikant,afgezet,1619,,--
freq,13,28871,318,369,28630,149


In [78]:
child_role.describe()

,drc_id,role_place,role_start_year,role_start_year_accuracy,role_remarks,role_type
count,3501,3501,3501,3501,3501,3501
unique,3272,730,319,2,2126,14
top,2210,,1780,,1774,emeritus_status
freq,4,1971,29,3469,18,2610


In [79]:
child_roles = pd.concat([child_role, role_minister], ignore_index=True)



In [80]:
child_roles.describe()

,drc_id,role_place,role_start_year,role_start_year_accuracy,role_remarks,role_type
count,32372,32372,30713,32372,32372,32372
unique,12548,8656,322,2,29264,15
top,21028,,1619,,--,predikant
freq,14,1983,382,32099,149,28871


In [81]:
child_roles.to_csv(folderlink+folder_output+'combined_child_roles.csv', sep=';', encoding='utf-8', index=False)